In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai

from fastai.io import *

from fastai.column_data import *
from fastai.text import *

import mygru

from pathlib import Path

/home/ejohnson/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Learning to generate Dickens-like text

### Setup

In [2]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH=Path('../data/dickens/')

TRN_PATH = 'trn'
VAL_PATH = 'val'
TRN = PATH / TRN_PATH
VAL = PATH / VAL_PATH

%ls {PATH}

davidcopperfield.txt  models/  tmp/  trn/  val/


In [34]:
TEXT = data.Field(lower=True, tokenize=list)
bs=2048; bptt=16; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(44, 53, 1, 1494913)

### Our model

In [35]:
gru = mygru.MyGRU
#gru = nn.GRU

class CharSequence(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.dropout = nn.Dropout(.1)
        self.rnn = gru(n_fac, n_hidden, nl, dropout=0.0)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.dropout(self.e(cs)), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = V(torch.zeros(self.nl, bs, n_hidden))

In [ ]:
char = CharSequence(md.nt, n_fac, n_hidden, 1)
if torch.cuda.is_available():
    char = char.cuda()
m = BasicModel(char)
learner = RNN_Learner(md, m, opt_fn=optim.Adam, crit=F.nll_loss)

#minimum_learning_rate_divisor = 1200
#percent_after_triangle_cycle = 15
#max_momentum=.97
#min_momentum=.85
#learner.fit(1e-2, 1, cycle_len=72, 
#           use_clr_beta=(minimum_learning_rate_divisor, 
#                         percent_after_triangle_cycle, 
#                         max_momentum, 
#                         min_momentum),
#                         wds=1e-5)
learner.fit(2e-2, 5, cycle_mult=2, cycle_len=1, 
           wds=1e-5)


epoch      trn_loss   val_loss   
    0      2.724434   2.515666  
    1      2.530044   2.403224  
    2      2.452712   2.383669  
    3      2.408054   2.337248  
    4      2.367373   2.309155  
    5      2.339531   2.295352  
    6      2.32204    2.291577  
    7      2.30302    2.254059  
    8      2.276722   2.217139  
    9      2.248484   2.18665   
    10     2.224118   2.169542  
    11     2.211261   2.153002  
    12     2.199884   2.157842  
    13     2.191732   2.153031  
    14     2.187825   2.148998  
    15     2.184781   2.139559  
    16     2.173614   2.123683  
    17     2.164717   2.111198  
    18     2.15509    2.104103  
    19     2.145673   2.098667  
    20     2.139509   2.089549  
    21     2.133651   2.094017  
    22     2.129979   2.081859  
    23     2.127539   2.084422  
    24     2.123623   2.07667   
    25     2.121288   2.069183  
    26     2.11953    2.078462  
    27     2.11682    2.068764  
 25%|██▌       | 11/44 [00:01<00:04,  7.85

### Test

In [22]:
def get_next(inp):
    device = None if torch.cuda.is_available() else -1
    idxs = TEXT.numericalize(inp, device=device, train=False)
    p = learner.model(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [23]:
get_next('for thos')
#TEXT

'k'

In [24]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [25]:
print(get_next_n('fourscore and seven years ago', 400))

fourscore and seven years agoldithig ayh hainksudng wion, stedonever.---at oadnede ly soult thicellteacce istouk awidthay drionly ad’ ourng theghing theen, rve achis ind ind eamer pes in as ittan isco rreutthake isos,  thave torr hegly, the ‘ile, saicasy wayt han  moreand aidg tiony uanl, da sthe min ag noter for mry seer erere--retad bouthimot ento wet toher and lwing iored loy the for angeupinth. g ivelly tor  mond barme of
